# Import Modules

In [20]:
#import libraries
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import math
import numpy as np
import pandas as pd
import tensorflow as ts
from scipy.stats import norm
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import r2_score

### Create Black Scholes for Euro Option

In [21]:
class EuropeanOptionBS:

    def __init__(self, S, K, T, r, q, sigma,Type):
        self.S = S
        self.K = K
        self.T = T
        self.r = r
        self.q = q        
        self.sigma = sigma
        self.Type = Type
        self.d1 = self.d1()
        self.d2 = self.d2()
        self.price = self.price()
        
    def d1(self):
        #
        d1 = (math.log(self.S / self.K) \
                   + (self.r - self.q + .5 * (self.sigma ** 2)) * self.T) \
                    / (self.sigma * self.T ** .5)       
        return d1

    def d2(self):
        d2 = self.d1 - self.sigma * self.T ** .5     
        return d2
    
    def price(self):
        if self.Type == "Call":
            price = self.S * math.exp(-self.q * self.T) * norm.cdf(self.d1) \
            - self.K * math.exp(-self.r *self.T) * norm.cdf(self.d2)
        if self.Type == "Put":
            price = self.K * math.exp(-self.r * self.T) * norm.cdf(-self.d2) \
            - self.S * math.exp(-self.q * self.T) * norm.cdf(-self.d1)            
        return price

## Create Dataset 

In [22]:
#dataset
r = np.arange(.0, .1, .01) #interest rates
Strike = np.arange(100, 235, 5) #strike price
T = np.arange(0.1, 2.1, 0.1) #time to maturity
sigma = np.arange(0.1, 2.1, .1) #volatility

data = []
for r_ in r:
    for Strike_ in Strike:
        for T_ in T:
            for sigma_ in sigma:
                data.append([r_, Strike_, T_, sigma_, \
                             EuropeanOptionBS(100, Strike_, T_, r_, 0, sigma_, "Call").price])
data = np.asarray(data)

#training and test datasets
X = data[:,:4] #params r, strike, T, sigma
y = data[:,4:5] #call price
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [23]:
## Define Artificial Neural Network

In [ ]:
#ANN with four layers, 10 neurons each
#activation function: ReLU
ANN = Sequential()
ANN.add(Dense(10,input_dim = 4, activation = 'relu'))
ANN.add(Dense(10, activation = 'relu'))
ANN.add(Dense(10, activation = 'relu'))
ANN.add(Dense(10, activation = 'relu'))         
ANN.add(Dense(1))
#Loss function = MSE, optimizer: Adam
ANN.compile(loss = 'mean_squared_error', optimizer='adam')
# fit the ANN on the training dataset
ANN.fit(X_train, y_train, epochs = 150, batch_size = 16)

Epoch 1/150
81000/81000 [==============================] - 15s 189us/step - loss: 157.3225
Epoch 2/150
81000/81000 [==============================] - 16s 192us/step - loss: 29.7270
Epoch 3/150
81000/81000 [==============================] - 16s 196us/step - loss: 28.4073
Epoch 4/150
81000/81000 [==============================] - 17s 207us/step - loss: 28.0536
Epoch 5/150
81000/81000 [==============================] - 21s 264us/step - loss: 27.8271
Epoch 6/150
81000/81000 [==============================] - 25s 306us/step - loss: 27.7409
Epoch 7/150
81000/81000 [==============================] - 25s 303us/step - loss: 27.3778
Epoch 8/150
81000/81000 [==============================] - 24s 293us/step - loss: 26.8116
Epoch 9/150
81000/81000 [==============================] - 24s 298us/step - loss: 25.7891
Epoch 10/150
81000/81000 [==============================] - 24s 298us/step - loss: 11.9200
Epoch 11/150
65504/81000 [=======================>......] - ETA: 4s - loss: 4.7813

In [ ]:
#prediction
y_pred = ANN.predict(X_test)

#Comparison real values and predictions on test dataset
plt.figure(figsize = (15,10))
plt.scatter(y_test, y_pred)
plt.xlabel("Real Value")
plt.ylabel("ANN Value")
plt.annotate("r-squared = {:.3f}".format(r2_score(y_test, y_pred)), (20, 1), size = 15)
plt.show();

In [ ]:
K = 170 #strike price
r = 0.05 #risk-free interest rate
sigma = .22 #implied volatiltiy
T = .5 #time to maturity
S = np.arange(50, 250, 1) #asset prices

PriceBS = [EuropeanOptionBS(S_, K, T, r, 0, sigma, "Call").price for S_ in S]
PriceANN = [S_ / 100 * \
            ANN.predict(np.array([[r, K / S_ * 100, T, sigma]]))[0][0] for S_ in S]

In [ ]:
#Comparison BS vs ANN prices
plt.figure(figsize = (15,10))
plt.plot(S, PriceBS, label = "Black-Scholes price")
plt.plot(S, PriceANN, label = "ANN price")
plt.legend(loc="upper left")
plt.xlabel("Asset Price")
plt.ylabel("Call Price")
plt.show()